In [1]:
def write(y,x='../data/gldas_dl.txt'):
    with open(x,'a') as f:
        f.write(y)
        f.write('\n')
        f.close()
    return _

In [2]:
lis = []
with open('../subset_GLDAS_NOAH025_M_2.1_20230123_143018_.txt') as f:
    for idx,x in enumerate(f):
        if idx == 0:
            continue
        lis.append(x.strip())
        # if idx == 2:
        #     break

In [6]:
print(lis[0])
print(lis[0][76:])


##therefore
ALPHA = 76

https://data.gesdisc.earthdata.nasa.gov/data/GLDAS/GLDAS_NOAH025_M.2.1/2000/GLDAS_NOAH025_M.A200001.021.nc4
GLDAS_NOAH025_M.A200001.021.nc4


In [7]:
import glob
x = glob.glob('../data/GLDAS/*.nc*')
print(x)
print(x[0][14:])

##therefore
BETA = 14

['../data/GLDAS/GLDAS_NOAH025_M.A200001.021.nc4']
GLDAS_NOAH025_M.A200001.021.nc4


In [9]:

import requests 

# # loads list of files to download
# lis = np.load('nclistnodupes.npy')[:3]


# this script obtained from NASA Earthdata's wiki
# https://wiki.earthdata.nasa.gov/
# please note that to run, you'll need to include your NASA Earthdata credentials

class SessionWithHeaderRedirection(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    # Overrides from the library to keep headers when redirected to or from the NASA auth host.
    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
               redirect_parsed.hostname != self.AUTH_HOST and \
               original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']
        return


# create session with the user credentials that will be used to authenticate access to the data

username="albertlarson" ###YOU THE READER OF FLUXTOFLOW NEED A NASA EARTHDATA ACCOUNT!!!
password="B6RXCmSnfViTtx5"
session = SessionWithHeaderRedirection(username, password)



# checks file folder to see if your destination folder for all these NLDAS files has any files in it, to
# prevent your code from downloading the same files multiple times.
destination_files = '../data/GLDAS/*.nc4'
y = [eeks[BETA:] for eeks in glob.glob(destination_files)]
not_dled = []
dled = []
for x in lis:
    if str(x[ALPHA:]) not in y:
        not_dled.append(x)
    if str(x[ALPHA:])  in y:
        dled.append(x)


# this while loop keeps attempting to download all of the files in your list of NLDAS files until all 
# of them have been downloaded. it also presents an http message number associated with your request 
# https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages
while True:
    for idx,i in enumerate(not_dled):
        try:

            # submit the request using the session
            response = session.get(i, stream=True)
            write(str(response.status_code))
            # raise an exception in case of http errors
            response.raise_for_status()
            # save the file
            with open(f'../data/GLDAS/{i[ALPHA:]}', 'wb') as fd:
                for data in response:
                    fd.write(data)
            fd.close()
        except requests.exceptions.HTTPError as e:
            # handle any errors here
            write(f"error: {e}")
    
    dled = []        
    not_dled = []
    for x in lis:
        y = [eeks[BETA:] for eeks in glob.glob(destination_files)]
        if str(x[ALPHA:]) not in y:
            not_dled.append(x)
        if str(x[ALPHA:]) in y:
            dled.append(x)
    write(f'dled: \t \n \t')
    for x in dled:
        write(f"\t {x} \t")
    write(f'not dled: \t \n \t')
    for x in not_dled:
        write(f"\t {x} \t")

            
    if len(not_dled) == 0:
        import sys
        sys.exit('no more left to download')

SystemExit: no more left to download

/work/albertl_uri_edu/.conda/envs/f2f_2/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
